In [30]:
from scipy.stats import ttest_ind, mannwhitneyu, shapiro
import numpy as np
import pandas as pd

In [3]:
data = pd.read_excel('dataset_hw8.xlsx')

data.head(10)

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0
5,2380,variant,0.0
6,2849,control,0.0
7,9168,control,0.0
8,6205,variant,0.0
9,7548,control,0.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       10000 non-null  int64  
 1   VARIANT_NAME  10000 non-null  object 
 2   REVENUE       10000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


In [5]:
data.describe()

,USER_ID,REVENUE
count,10000.000000,10000.000000
mean,4981.080200,0.099447
std,2890.590115,2.318529
min,2.000000,0.000000
25%,2468.750000,0.000000
50%,4962.000000,0.000000
75%,7511.500000,0.000000
max,10000.000000,196.010000


In [6]:
data.isna().sum()

USER_ID         0
VARIANT_NAME    0
REVENUE         0
dtype: int64

In [10]:
data.VARIANT_NAME.value_counts()

variant    5016
control    4984
Name: VARIANT_NAME, dtype: int64

> В вариантах всего два значения - variant и control - это хорошо.

In [13]:
data[data['REVENUE']>0]

,USER_ID,VARIANT_NAME,REVENUE
13,2529,variant,2.15
49,6429,control,3.25
139,3331,variant,4.27
149,8110,variant,1.75
152,8607,control,2.99
...,...,...,...
9651,9928,variant,1.25
9711,6468,control,2.93
9724,5205,control,14.17
9725,702,variant,2.92


> Данных с выручкой крайне мало.

In [7]:
data.USER_ID.count()

10000

In [9]:
data.USER_ID.nunique()

6324

> Уникальных пользователей меньше общего количества, значит есть повторы. Проверим что один и тот же пользователь не попал в обе группы.

In [14]:
variants = data.groupby('USER_ID', as_index=False).agg({'VARIANT_NAME': pd.Series.nunique})

In [15]:
variants.head(10)

,USER_ID,VARIANT_NAME
0,2,1
1,3,2
2,4,1
3,5,1
4,6,1
5,9,1
6,10,2
7,11,1
8,12,1
9,13,1


> Видим что есть одни и те же пользователи в обеих группах.

In [17]:
more_than_one_group = variants.query('VARIANT_NAME > 1')

In [18]:
data_new = data[~data.USER_ID.isin(more_than_one_group.USER_ID)].sort_values('USER_ID')

In [19]:
data_new.shape[0], data.shape[0]

(6070, 10000)

In [20]:
control = data_new.query('VARIANT_NAME == "control"')
control.shape[0]

3026

In [21]:
test = data_new.query('VARIANT_NAME == "variant"')
test.shape[0]

3044

> Итак, мы выкинули более трети данных. То что пользователи были в обеих группах говорит о том что есть проблемы со сплит системой, скорее всего один и тот же пользователь входил с разных устройств и мог попадать как в тестовую так и контрольную группу. 

> После разделения оставшихся данных видно, что группы примерно одинаковые.

> Посмотрим на распределение в группах:

In [23]:
alpha = 0.05

st_a = shapiro(control.REVENUE)
print('Distribution is {}normal\n'.format({True:'not ', False:''}[st_a[1] < alpha]))

Distribution is not normal



In [24]:
st_b = shapiro(control.REVENUE)
print('Distribution is {}normal\n'.format({True:'not ', False:''}[st_b[1] < alpha]))

Distribution is not normal



In [27]:
ttest_ind(control.REVENUE.values, test.REVENUE.values, equal_var = False)

Ttest_indResult(statistic=1.403762252371675, pvalue=0.1604844017690842)

> pvalue > 0.05, значит между выборками нет стат. значимых различий. 

In [29]:
st, pval = mannwhitneyu(control.REVENUE.values, test.REVENUE.values)
pval

0.2444173738649208

> p-value по тесту Манна-Уитни тоже больше 0.05, что подтверждает что между выборками нет стат. значимых различий.

> В общем, если закрыть глаза на кривые данные, то менеджеру можно сказать что нужно остаться при старой версии. А лучше всего переделать тест, и дождаться его окончания прежде чем анализировать и делать выводы по полученому результату.